In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine

# Mac Users

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

In [3]:
# /usr/local/bin/chromedriver

In [4]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

# Windows

In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
url = 'https://www.imdb.com/search/title/?companies=co0144901&ref_=adv_prv'
browser.visit(url)

In [7]:
shows_list = []
count = 0
for x in range(1, 264):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    articles = soup.find_all('div', class_='lister-item-content')
    
 
    
    for article in articles:
        
        
       
        # Use Beautiful Soup's find() method to navigate and retrieve attributes
        h3 = article.find('h3')
        
        title = h3.find('a').text

        
        try:
            rating_bar = article.find('div', class_='ratings-bar')
            rating = rating_bar.find('div', class_='inline-block ratings-imdb-rating')['data-value']

            count+=1   
        except:
            rating = None
        shows_dict = {"title":title, "rating":rating}
        shows_list.append(shows_dict)
            

        
    try:
        element_next=browser.links.find_by_partial_text('Next')
        element_next.click()
          
    except:
        print("Scraping Complete")
print(len(shows_list))

Scraping Complete
13128


In [8]:
#converting show list into DataFrame
raw_df = pd.DataFrame(shows_list)
raw_df

,title,rating
0,Lucifer,8.2
1,The Umbrella Academy,8
2,Project Power,6
3,Enola Holmes,None
4,Cobra Kai,8.8
...,...,...
13123,Derry Girls,9
13124,Derry Girls,8.5
13125,Supergirl,7.4
13126,Supergirl,7.5


In [9]:
#Dropping rows with None value
shows_df = raw_df.dropna()

In [10]:
#Converting the rating into float type
shows_df['rating'] = shows_df['rating'].astype(float)
shows_df.dtypes

C:\Users\spuna\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


title      object
rating    float64
dtype: object

In [16]:
shows_df.head()

,title,rating
0,Lucifer,8.2
1,The Umbrella Academy,8.0
2,Project Power,6.0
4,Cobra Kai,8.8
5,The Sleepover,5.5


In [17]:
#using strip function to remove extra space
shows_df["title"] = shows_df["title"].str.strip()

C:\Users\spuna\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
#grouping titles and calculating avg rating
grouped_shows_df = shows_df.groupby('title')
avg_rating = grouped_shows_df['rating'].mean()
#formatting
avg_rating = avg_rating.map("{:.1f}".format)

In [19]:
netflixrating_final = pd.DataFrame(avg_rating)
netflixrating_final = netflixrating_final.reset_index('title')

In [20]:
netflixrating_final.head()

,title,rating
0,#Alive,6.2
1,#Anne Frank Parallel Stories,6.4
2,#BlackAF,6.7
3,#HappyBirthdaySense8,8.9
4,#REALITYHIGH,5.2


# Netflix shows file transformation

In [23]:
netflix_titles = "Resources/netflix_titles_raw.csv"

In [24]:
netflix_df = pd.read_csv(netflix_titles)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [25]:
netflix_tv = netflix_df.loc[netflix_df["type"] == "TV Show",:]

In [26]:
# extracting desired columns
netflix_new = netflix_tv[["type","title","country","release_year","listed_in"]]

In [27]:
netflix_shows = netflix_new.drop_duplicates("title")
print(netflix_shows.count())
netflix_shows.head()

type            1958
title           1958
country         1677
release_year    1958
listed_in       1958
dtype: int64


,type,title,country,release_year,listed_in
2,TV Show,Transformers Prime,United States,2013,Kids' TV
3,TV Show,Transformers: Robots in Disguise,United States,2016,Kids' TV
5,TV Show,Apaches,Spain,2016,"Crime TV Shows, International TV Shows, Spanis..."
8,TV Show,Fire Chasers,United States,2017,"Docuseries, Science & Nature TV"
26,TV Show,Castle of Stars,NaN,2015,"International TV Shows, Romantic TV Shows, TV ..."


In [28]:
# renaming the column
netflix_tvshows = netflix_shows.rename(columns={"listed_in":"category"})

# Create database connection

In [29]:
import pandas as pd
from sqlalchemy import create_engine

In [30]:
connection_string = "postgres:postgres@localhost:5432/netflix_db"
engine = create_engine(f'postgresql://{connection_string}')

In [35]:
# Confirm tables
engine.table_names()

['netflix_rating', 'netflix_shows']

# Load DataFrames into database

In [32]:
netflixrating_final.to_sql(name='netflix_rating', con=engine, if_exists='append', index=True)

In [33]:
netflix_tvshows.to_sql(name='netflix_shows', con=engine, if_exists='append', index=True)